##### **Création d'une fonction pour charger les données dans des dataframes à partir d'une URL**

In [2]:
import requests
import zipfile
from io import BytesIO
import pandas as pd

def download_excel_to_dataframe(url, excel_file_name=None, sheet_name=None):
    try:
        # Télécharger le fichier
        response = requests.get(url)
        response.raise_for_status()
        
        # Vérifier si c'est un fichier ZIP
        content_type = response.headers.get('Content-Type', '').lower()
        if 'zip' in content_type or url.endswith('.zip'):
            print("Le fichier téléchargé est un fichier ZIP.")
            zip_content = BytesIO(response.content)
            with zipfile.ZipFile(zip_content) as zip_file:
                # Si aucun nom de fichier Excel n'est spécifié, utiliser le premier fichier Excel trouvé
                if excel_file_name is None:
                    excel_file_name = next((f for f in zip_file.namelist() if f.endswith('.xlsx')), None)
                    if excel_file_name is None:
                        raise FileNotFoundError("Aucun fichier Excel (.xlsx) n'a été trouvé dans le ZIP.")
                    print(f"Fichier Excel non spécifié, utilisation de : {excel_file_name}")
                
                # Vérifier si le fichier Excel spécifié existe dans le ZIP
                if excel_file_name not in zip_file.namelist():
                    raise FileNotFoundError(f"Le fichier '{excel_file_name}' est introuvable dans le ZIP.")
                
                # Extraire le fichier Excel dans un objet BytesIO
                with zip_file.open(excel_file_name) as excel_file:
                    excel_data = pd.ExcelFile(BytesIO(excel_file.read()))  # Lire tout le fichier en mémoire
        
        # Si ce n'est pas un ZIP, on suppose que c'est un fichier Excel
        elif 'excel' in content_type or url.endswith('.xlsx'):
            print("Le fichier téléchargé est un fichier Excel.")
            excel_data = pd.ExcelFile(BytesIO(response.content))
        
        else:
            raise ValueError("Le fichier téléchargé n'est ni un fichier Excel, ni un fichier ZIP.")
        
        # Utiliser la première feuille si non spécifié
        if sheet_name is None:
            sheet_name = excel_data.sheet_names[0]
            print(f"Feuille non spécifiée, chargement de : {sheet_name}")
        
        # Charger la feuille dans un DataFrame
        df = excel_data.parse(sheet_name)
        print(f"Feuille '{sheet_name}' chargée.")
        
        return df
    
    except Exception as e:
        print(f"Erreur : {e}")
        return None

##### **Chômage**

In [58]:
chômage = download_excel_to_dataframe("https://www.insee.fr/fr/statistiques/fichier/2532173/econ-gen-taux-cho-trim-2.xlsx")
chômage

Le fichier téléchargé est un fichier Excel.
Feuille non spécifiée, chargement de : Données
Feuille 'Données' chargée.


,Taux de chômage selon le sexe et l'âge,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 190,Unnamed: 191,Unnamed: 192,Unnamed: 193,Unnamed: 194,Unnamed: 195,Unnamed: 196,Unnamed: 197,Unnamed: 198,Unnamed: 199
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,en %,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sexe et âge,1975-T1,1975-T2,1975-T3,1975-T4,1976-T1,1976-T2,1976-T3,1976-T4,1977-T1,...,2022-T2,2022-T3,2022-T4,2023-T1,2023-T2,2023-T3,2023-T4,2024-T1,2024-T2,2024-T3
3,Femmes,4.6,4.9,5.2,5.5,5.8,5.9,5.8,5.8,6.1,...,7.2,7,6.9,6.8,7.1,7.4,7.4,7.3,7.4,7.2
4,15-24 ans,9.8,10.4,11.1,11.8,12.8,13.1,12.7,12.6,13.5,...,17.3,15.7,14.8,15.6,16,16,16.7,17,18.2,20
5,25-49 ans,3.3,3.5,3.7,3.9,4,4,4,4,4.1,...,6.6,6.4,6.4,6.3,6.6,7.2,7.1,7,6.8,6.3
6,50 ans ou plus,2.1,2.2,2.4,2.5,2.6,2.7,2.8,3,3.2,...,5,5.2,5.2,5,5.1,5.2,4.9,4.9,5,4.8
7,Hommes,2.3,2.6,2.8,2.9,2.9,2.8,2.7,2.7,2.9,...,7.6,7.5,7.3,7.4,7.4,7.4,7.7,7.6,7.3,7.6
8,15-24 ans,4.8,5.4,5.8,6,6,5.8,5.6,5.9,6.6,...,18.9,18.9,18.7,17.8,18.1,18.4,18.3,19.3,17.7,19.4
9,25-49 ans,1.7,1.9,2.1,2.1,2.1,2,1.9,1.9,2,...,6.8,6.5,6.5,6.4,6.6,6.4,6.9,6.6,6.5,6.9


In [59]:
chômage.columns = chômage.iloc[2] 
chômage = chômage.drop(index=list(range(0, 2)) + list(range(3, 11)) + list(range(12, 21)))
chômage = chômage.drop(index=2) 
chômage.rename(columns={"Sexe et âge": "TrimAnnee"}, inplace=True)
chômage

2,TrimAnnee,1975-T1,1975-T2,1975-T3,1975-T4,1976-T1,1976-T2,1976-T3,1976-T4,1977-T1,...,2022-T2,2022-T3,2022-T4,2023-T1,2023-T2,2023-T3,2023-T4,2024-T1,2024-T2,2024-T3
11,Ensemble,3.2,3.5,3.8,3.9,4,4,3.9,4,4.2,...,7.4,7.2,7.1,7.1,7.2,7.4,7.5,7.5,7.3,7.4


##### **Confiance**

In [63]:
confiance = download_excel_to_dataframe("https://www.insee.fr/en/statistiques/serie/telecharger/xlsx/001587668?ordre=antechronologique&transposition=donneesligne&periodeDebut=10&anneeDebut=1972&periodeFin=11&anneeFin=2024&revision=sansrevisions")
confiance

Le fichier téléchargé est un fichier ZIP.
Fichier Excel non spécifié, utilisation de : serie_001587668_25122024.xlsx
Feuille non spécifiée, chargement de : monthly_values
Feuille 'monthly_values' chargée.


c:\Users\Carla Lucas\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Label,idBank,Last update,Period,2024-11,2024-10,2024-09,2024-08,2024-07,2024-06,...,1973-07,1973-06,1973-05,1973-04,1973-03,1973-02,1973-01,1972-12,1972-11,1972-10
0,Monthly consumer confidence survey - Summary i...,1587668,27/11/2024 08:45,NaN,90.0,93.0,95.0,92.0,91.0,90.0,...,127.0,129.0,131.0,131.0,131.0,130.0,130.0,129.0,128.0,127.0


In [64]:
confiance.drop(columns=["Label","idBank", "Last update", "Period"])

,2024-11,2024-10,2024-09,2024-08,2024-07,2024-06,2024-05,2024-04,2024-03,2024-02,...,1973-07,1973-06,1973-05,1973-04,1973-03,1973-02,1973-01,1972-12,1972-11,1972-10
0,90.0,93.0,95.0,92.0,91.0,90.0,90.0,90.0,91.0,90.0,...,127.0,129.0,131.0,131.0,131.0,130.0,130.0,129.0,128.0,127.0


On produit à partir de l'indice mensuel de confiance un indice trimestriel moyen 

In [65]:
# On transpose pour travailler sur les index
conf_transpose = confiance.T
conf_transpose.columns = ['Value']  

# On exclue les valeurs qui ne respectent pas le format AAAA-MM
conf_transpose = conf_transpose[conf_transpose.index.str.match(r'^\d{4}-\d{2}$')]

# On extrait l'année et le mois depuis l'index
conf_transpose['Annee'] = conf_transpose.index.str[:4].astype(int)
conf_transpose['Mois'] = conf_transpose.index.str[-2:].astype(int)

# Création d'une colonne pour le trimestre
conf_transpose['TrimAnnee'] = ((conf_transpose['Mois'] - 1) // 3) + 1

# Calcul des moyennes trimestrielles en groupant par année et trimestre
TrimAnnee_Moyen = (
    conf_transpose
    .groupby(['Annee', 'TrimAnnee'])['Value']
    .mean()
    .reset_index()
)

# Créeation d'un dataframe avec nos données
TrimAnnee_Moyen['Trimestre'] = TrimAnnee_Moyen['Annee'].astype(str) + '-T' + TrimAnnee_Moyen['TrimAnnee'].astype(str)
conf_moyenne = TrimAnnee_Moyen[['Trimestre', 'Value']].rename(columns={'Value': 'Average'})
conf_moyenne


,Trimestre,Average
0,1972-T4,128.000000
1,1973-T1,130.333333
2,1973-T2,130.333333
3,1973-T3,126.000000
4,1973-T4,122.000000
...,...,...
204,2023-T4,87.666667
205,2024-T1,90.666667
206,2024-T2,90.000000
207,2024-T3,92.666667


In [45]:
pib = download_excel_to_dataframe("https://www.insee.fr/fr/statistiques/fichier/2830547/econ-gen-pib-composante-trim.xlsx")
pib

Le fichier téléchargé est un fichier Excel.
Feuille non spécifiée, chargement de : Données
Feuille 'Données' chargée.


,Évolution du produit intérieur brut et de ses composantes,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,par rapport au trimestre précédent en volume en %,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Trimestre,Produit intérieur brut (PIB),Importations,Dépense de consommation des ménages,Dépense de consommation des APU1,Formation brute de capital fixe,dont :,NaN,NaN,Exportations,Contributions :,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Entreprises non financières,ménages,APU1,NaN,Demande intérieure finale hors stocks,Variations de stocks,Commerce extérieur
4,2024-T3,0.38673,-0.589976,0.610828,0.561648,-0.706234,-1.228128,-0.645364,0.539162,-0.758542,0.305012,0.134889,-0.053171
...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,1. Administrations publiques.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,Note : données révisées ; les volumes sont mes...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,"Lecture : au 3e trimestre 2024, le produit int...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309,Champ : France.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
pib.columns = pib.iloc[2] 
pib = pib[["Trimestre", "Produit intérieur brut (PIB)"]]
pib = pib.drop(index=range(0,4))
pib = pib.drop(index=range(306,311))
pib

2,Trimestre,Produit intérieur brut (PIB)
4,2024-T3,0.38673
5,2024-T2,0.184356
6,2024-T1,0.208671
7,2023-T4,0.447218
8,2023-T3,0.050172
...,...,...
301,1950-T2,2.525426
302,1950-T1,2.748324
303,1949-T4,1.397159
304,1949-T3,1.49785


In [66]:
ipc = download_excel_to_dataframe("https://www.insee.fr/fr/statistiques/serie/telecharger/xlsx/001759970?ordre=antechronologique&transposition=donneescolonne&periodeDebut=1&anneeDebut=1990&periodeFin=11&anneeFin=2024&revision=sansrevisions")
ipc

Le fichier téléchargé est un fichier ZIP.
Fichier Excel non spécifié, utilisation de : serie_001759970_25122024.xlsx
Feuille non spécifiée, chargement de : valeurs_mensuelles
Feuille 'valeurs_mensuelles' chargée.


c:\Users\Carla Lucas\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Libellé,Indice des prix à la consommation - Base 2015 - Ensemble des ménages - France - Ensemble,Unnamed: 2
0,idBank,001759970,NaN
1,Dernière mise à jour,13/12/2024 08:45,NaN
2,Période,NaN,Date de parution au JO
3,2024-11,119.72,15/12/2024
4,2024-10,119.89,17/11/2024
...,...,...,...
417,1990-05,67.19,29/06/1990
418,1990-04,67.09,01/06/1990
419,1990-03,66.72,02/05/1990
420,1990-02,66.56,30/03/1990


In [67]:
ipc = ipc[["Libellé", "Indice des prix à la consommation - Base 2015 - Ensemble des ménages - France - Ensemble"]]
ipc = ipc.drop(index=range(0,3))
ipc

,Libellé,Indice des prix à la consommation - Base 2015 - Ensemble des ménages - France - Ensemble
3,2024-11,119.72
4,2024-10,119.89
5,2024-09,119.56
6,2024-08,121.06
7,2024-07,120.42
...,...,...
417,1990-05,67.19
418,1990-04,67.09
419,1990-03,66.72
420,1990-02,66.56


In [68]:
ipc

,Libellé,Indice des prix à la consommation - Base 2015 - Ensemble des ménages - France - Ensemble
3,2024-11,119.72
4,2024-10,119.89
5,2024-09,119.56
6,2024-08,121.06
7,2024-07,120.42
...,...,...
417,1990-05,67.19
418,1990-04,67.09
419,1990-03,66.72
420,1990-02,66.56


In [70]:
# Étape 1 : Extraire l'année et le mois
ipc['Année'] = ipc['Libellé'].str[:4].astype(int)
ipc['Mois'] = ipc['Libellé'].str[5:].astype(int)

# Étape 2 : Déterminer le trimestre
ipc['Trimestre'] = ((ipc['Mois'] - 1) // 3) + 1

# Étape 3 : Calculer la moyenne trimestrielle
# Créer une colonne clé 'Trimestre'
ipc['Trimestre'] = ipc['Année'].astype(str) + '-T' + ipc['Trimestre'].astype(str)

# Calcul de la moyenne par 'Trimestre'
moy_ipc = ipc.groupby('Trimestre')['Indice des prix à la consommation - Base 2015 - Ensemble des ménages - France - Ensemble'].mean().reset_index()

moy_ipc

,Trimestre,Indice des prix à la consommation - Base 2015 - Ensemble des ménages - France - Ensemble
0,1990-T1,66.566667
1,1990-T2,67.153333
2,1990-T3,67.653333
3,1990-T4,68.330000
4,1991-T1,68.803333
...,...,...
135,2023-T4,118.350000
136,2024-T1,118.956667
137,2024-T2,120.126667
138,2024-T3,120.346667
